In [16]:
import re 
import os
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import csv
import glob
import os
from matplotlib.animation import FuncAnimation
from IPython import display
import imageio


### Let us get all the files 

In [ ]:
path = "./Data/*/"#/*/foragers100/*/*_dis.txt"
paths = ["foragers100/beta1.5/", "foragers100/beta2.5/", "foragers100/beta3.5/", "foragers100/beta4.5/"]
path_rad = ["radius1/", "radius0.1/", "radius0.01/", "radius0.001/"]
all_files = [] 

for p in paths : 
    for r in path_rad : 
        res = [f for f in os.listdir(path+p+r) if re.search(r'([0]).txt$', f)]
        all_files.append(res)
    
for f in all_files[0]:
    for p in paths : 
        for r in path_rad:
            print(path+p+r+f)
         

### Now let us construct a dataframe for these files such that we have the x y coordinates of every forager at every time step for each file

In [8]:
df_tet = []
for csv in all_files[0]:
    for p in paths :
        for r in path_rad:
            i = 0 
            while i < 99 :
                frame = pd.read_csv(path+p+r+csv, sep=',', engine='python',usecols=[0+(i*4),1+(i*4),2+(i*4),3+(i*4)], names=["id", "x", "y", "k"])
                frame['filename'] = os.path.basename(csv)
                frame['beta'] = p.split('beta')[1].split('/')[0]
                frame['radius'] = r.split('s')[1].split('/')[0]
                df_tet.append(frame)

                i = i + 1
            
df_tet = pd.concat(df_tet, ignore_index=True)
            

In [9]:
df_tet

,id,x,y,k,filename,beta,radius
0,0,0.250684,0.658152,2,0.txt,1.5,1
1,0,0.250684,0.658152,1,0.txt,1.5,1
2,0,0.249803,0.656149,598,0.txt,1.5,1
3,0,0.249803,0.656149,597,0.txt,1.5,1
4,0,0.249803,0.656149,596,0.txt,1.5,1
...,...,...,...,...,...,...,...
158395,98,0.995133,0.803153,-1,0.txt,4.5,0.001
158396,98,0.992019,0.803600,-1,0.txt,4.5,0.001
158397,98,0.989691,0.808163,-1,0.txt,4.5,0.001
158398,98,0.989490,0.808101,-1,0.txt,4.5,0.001


### This function will read in the dataframe

In [12]:
def plot_for_offset(i,file,beta,radius):
    print(i)
    plt.gca().cla()
    display.clear_output(wait=False)
    j=0
    df2 = df_tet.loc[(df_tet['filename'] == file) & (df_tet['beta'] == beta) & (df_tet['radius'] == radius) ]

    for j in range(99) :
        
        df3 = df2.loc[(df2['id'] == j) ]
        df3 = df3.reset_index()
        ax.scatter(x=df3.loc[i]['x'], y=df3.loc[i]['y'], color=cmap(j))
        ax.plot(df3.loc[:i+1]['x'], df3.loc[:i+1]['y'], color=cmap(j))
        fig.canvas.draw()

    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    return img

### this function will make the plot and save it as a GIF 

In [11]:
def make_plot(file,beta,radius):
    
    i = 0
    nametosave = 'tethered'+"_"+beta+"_"+radius
    allimg = [plot_for_offset(i,file,beta,radius) for i in range(99)]
    imageio.mimsave(nametosave+".gif", allimg , fps=5)
    plt.cla()
    fig.clear()
    del allimg
    

In [15]:
def get_cmap(n, name='viridis'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

### Now we can use these functions 
### it runs a loop over all betas and radii and calls the above function to run on the dataframe

In [ ]:
betas = ['1.5' ,'2.5', '3.5', '4.5']
rads = ['1', '0.1', '0.01', '0.001']
for b in betas:
    for r in rads :
#         fig.clear()
        
        fig, ax = plt.subplots()
        fig.suptitle("beta"+b+"|"+"radius"+r, fontsize=20)
        ax.set(xlim=(0, 1), ylim=(0, 1))
        fig.set_figheight(10)
        fig.set_figwidth(10)

        cmap = get_cmap(100)
        print(r, b)
        make_plot('0.txt', b, r)